## Mounting the Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/END

/content/drive/MyDrive/END


In [4]:
%%bash
python -m spacy download en
python -m spacy download de
python -m spacy download fr

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/fr_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/fr
You can now load the model via spacy.load('fr')


In [20]:
!pip install google-trans-new

## Importing Packages

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from google_trans_new import google_translator
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [7]:
#Setting the manual seeds
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Loading the spacy language models (German and French) for tokenization

In [8]:
spacy_fr = spacy.load('fr')
spacy_de = spacy.load('de')

In [9]:
%cd /content/drive/MyDrive/END/multi30k-dataset

/content/drive/MyDrive/END/multi30k-dataset


In [10]:
!git pull

Already up to date.


In [11]:
%cd ..

/content/drive/My Drive/END


## Tokenizing the text and creating Fields for Source and Target languages

In [12]:
## Feeding the reversed input sentence
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) 
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_fr(text):
    """
    Tokenizes French text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_fr.tokenizer(text)][::-1]

In [13]:
SRC = Field(tokenize = tokenize_fr, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

## Creating Train, Validation and Test splits

In [14]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.fr', '.de'), root="/content/drive/MyDrive/END/data",
                                                    fields = (SRC, TRG))

In [15]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


## Visualizing some examples

In [43]:
translator = google_translator()
source_ex = train_data.examples[0].src[::-1]
target_ex = train_data.examples[0].trg

source_eng_translation = translator.translate(' '.join(source_ex),lang_tgt='en')
target_eng_translation = translator.translate(' '.join(target_ex),lang_tgt='en')

print(f"\nSouce Sentence >>>>> {' '.join(source_ex)}")
print(f"Target Sentence >>>>> {' '.join(target_ex)}")

print(f"\nSouce Sentence translated to English >>>>> {source_eng_translation}")
print(f"Target Sentence translated to English >>>>> {target_eng_translation}")


Souce Sentence >>>>> deux jeunes hommes blancs sont dehors près de buissons .
Target Sentence >>>>> zwei junge weiße männer sind im freien in der nähe vieler büsche .

Souce Sentence translated to English >>>>> two young white men are outside near the bushes. 
Target Sentence translated to English >>>>> two young white men are outdoors near many bushes. 


## Building Vocabulary for source and target documents

In [38]:
# Excluding tokens with less than 2 occurences
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [39]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 6462
Unique tokens in target (en) vocabulary: 7855


### Lets see some words and corresponding integer representation for both vocabs

In [47]:
# Source Example
for token in source_ex:
  print(f"Token ---> {token} ; Integer_Rep ---> {SRC.vocab.stoi[token]}")

Token ---> deux ; Integer_Rep ---> 21
Token ---> jeunes ; Integer_Rep ---> 81
Token ---> hommes ; Integer_Rep ---> 33
Token ---> blancs ; Integer_Rep ---> 213
Token ---> sont ; Integer_Rep ---> 28
Token ---> dehors ; Integer_Rep ---> 88
Token ---> près ; Integer_Rep ---> 70
Token ---> de ; Integer_Rep ---> 7
Token ---> buissons ; Integer_Rep ---> 1180
Token ---> . ; Integer_Rep ---> 5


In [48]:
# target Example
for token in target_ex:
  print(f"Token ---> {token} ; Integer_Rep ---> {TRG.vocab.stoi[token]}")

Token ---> zwei ; Integer_Rep ---> 18
Token ---> junge ; Integer_Rep ---> 26
Token ---> weiße ; Integer_Rep ---> 253
Token ---> männer ; Integer_Rep ---> 30
Token ---> sind ; Integer_Rep ---> 84
Token ---> im ; Integer_Rep ---> 20
Token ---> freien ; Integer_Rep ---> 88
Token ---> in ; Integer_Rep ---> 7
Token ---> der ; Integer_Rep ---> 15
Token ---> nähe ; Integer_Rep ---> 110
Token ---> vieler ; Integer_Rep ---> 7649
Token ---> büsche ; Integer_Rep ---> 3171
Token ---> . ; Integer_Rep ---> 4


## Creating the DataLoaders/Iterators

In [50]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [51]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

## Building the Seq2Seq Model



### Encoder

In [52]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        # Here you can use nn.GRU(emb_dim, hid_dim) or 
        # nn.RNN(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

### Decoder


In [53]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

### Seq2Seq

In [54]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

# Training the Seq2Seq Model


In [55]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 3
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

#### Initializing the Model Weights

In [56]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(6462, 256)
    (rnn): LSTM(256, 512, num_layers=3, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=3, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=7855, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [57]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 19,253,679 trainable parameters


In [59]:
optimizer = optim.AdamW(model.parameters())

In [60]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

### Train and Evaluate Function

In [61]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [62]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [63]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### Model Training

In [64]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 53s
	Train Loss: 5.224 | Train PPL: 185.735
	 Val. Loss: 4.992 |  Val. PPL: 147.292
Epoch: 02 | Time: 0m 54s
	Train Loss: 4.772 | Train PPL: 118.193
	 Val. Loss: 5.053 |  Val. PPL: 156.512
Epoch: 03 | Time: 0m 53s
	Train Loss: 4.484 | Train PPL:  88.568
	 Val. Loss: 4.850 |  Val. PPL: 127.725
Epoch: 04 | Time: 0m 54s
	Train Loss: 4.258 | Train PPL:  70.657
	 Val. Loss: 4.709 |  Val. PPL: 110.974
Epoch: 05 | Time: 0m 53s
	Train Loss: 4.058 | Train PPL:  57.833
	 Val. Loss: 4.590 |  Val. PPL:  98.480
Epoch: 06 | Time: 0m 54s
	Train Loss: 3.934 | Train PPL:  51.116
	 Val. Loss: 4.564 |  Val. PPL:  95.990
Epoch: 07 | Time: 0m 54s
	Train Loss: 3.826 | Train PPL:  45.872
	 Val. Loss: 4.494 |  Val. PPL:  89.488
Epoch: 08 | Time: 0m 54s
	Train Loss: 3.722 | Train PPL:  41.339
	 Val. Loss: 4.431 |  Val. PPL:  84.031
Epoch: 09 | Time: 0m 54s
	Train Loss: 3.635 | Train PPL:  37.910
	 Val. Loss: 4.433 |  Val. PPL:  84.186
Epoch: 10 | Time: 0m 53s
	Train Loss: 3.580 | Train PPL

We'll load the parameters (`state_dict`) that gave our model the best validation loss and run it the model on the test set.

In [65]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.230 | Test PPL:  68.727 |


### Lets see how our model performs on one example from the test set

In [66]:
one_test_batch = list(test_iterator)[0]

In [67]:
one_test_batch.src.shape, one_test_batch.trg.shape

(torch.Size([15, 128]), torch.Size([13, 128]))

In [84]:
src_ex = one_test_batch.src[ :, 0]
trg_ex = one_test_batch.trg[ :, 0]
src_sentence = src_ex.unsqueeze(1)

source_sentence = " ".join([SRC.vocab.itos[x] for x in src_ex.cpu()][::-1])
target_sentence = " ".join([TRG.vocab.itos[x] for x in trg_ex.cpu()])
model_prediction = model(src_sentence, src_sentence, 0)
model_prediction = model_prediction[1:].view(-1, model_prediction.shape[-1])
model_prediction = " ".join([TRG.vocab.itos[x] for x in torch.argmax(model_prediction, axis=1).cpu()])

source_eng_translation = translator.translate(source_sentence,lang_tgt='en')
target_eng_translation = translator.translate(target_sentence,lang_tgt='en')
pred_eng_translation = translator.translate(model_prediction.split('.')[0], lang_tgt='en')
print(f"\nSouce Sentence >>>>> {source_sentence}")
print(f"Target Sentence >>>>> {target_sentence}")
print(f"Predicted Target Sentence >>>>> {model_prediction}")

print(f"\nSouce Sentence translated to English >>>>> {source_eng_translation}")
print(f"Target Sentence translated to English >>>>> {target_eng_translation}")
print(f"Predicted Target Sentence translated to English >>>>> {pred_eng_translation}")


Souce Sentence >>>>> <pad> <pad> <pad> <pad> <eos> une petite fille ouvrant un cadeau de noël . <sos>
Target Sentence >>>>> <sos> ein kleines mädchen , das ein <unk> auspackt . <eos> <pad> <pad>
Predicted Target Sentence >>>>> ein kleines mädchen mädchen spielt . <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>

Souce Sentence translated to English >>>>> <pad> <pad> <pad> <pad> <eos> a little girl opening a Christmas present. <sos> 
Target Sentence translated to English >>>>> <sos> a little girl unpacking an. <eos> <pad> <pad> 
Predicted Target Sentence translated to English >>>>> a little girl girl plays 
